<a href="https://colab.research.google.com/github/HarshaR99/Colab_python/blob/master/Linear_Regression_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%tensorflow_version 2.x

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output
from six.moves import urllib

import tensorflow.compat.v2.feature_column as fc

import tensorflow as tf

# Loading Data set

The data is about the information of the passengers of the Titanic

In [0]:
# Load dataset.
dftrain = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/train.csv') # training data
dfeval = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/eval.csv') # testing data
y_train = dftrain.pop('survived')
y_eval = dfeval.pop('survived')

# Creating feature columns

One thing I found is that  we are just passing attributes of the dataframe .
We are trying to map string datatypes to numeric type and and passing the name of the attributes of nmeric and categorial columns

In [0]:
CATEGORICAL_COLUMNS = ['sex', 'n_siblings_spouses', 'parch', 'class', 'deck',
                       'embark_town', 'alone']
NUMERIC_COLUMNS = ['age', 'fare']

feature_columns = []
for feature_name in CATEGORICAL_COLUMNS:
  vocabulary = dftrain[feature_name].unique()
  feature_columns.append(tf.feature_column.categorical_column_with_vocabulary_list(feature_name,vocabulary))

for feature_name in NUMERIC_COLUMNS:
    feature_columns.append(tf.feature_column.numeric_column(feature_name,dtype=tf.float32))

In [0]:
# what unique function does
dftrain['class'].unique()

array(['Third', 'First', 'Second'], dtype=object)

In [0]:
tf.feature_column.categorical_column_with_vocabulary_list('class',dftrain['class'].unique())

VocabularyListCategoricalColumn(key='class', vocabulary_list=('Third', 'First', 'Second'), dtype=tf.string, default_value=-1, num_oov_buckets=0)

In [0]:
print(feature_columns)

[VocabularyListCategoricalColumn(key='sex', vocabulary_list=('male', 'female'), dtype=tf.string, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='n_siblings_spouses', vocabulary_list=(1, 0, 3, 4, 2, 5, 8), dtype=tf.int64, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='parch', vocabulary_list=(0, 1, 2, 5, 3, 4), dtype=tf.int64, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='class', vocabulary_list=('Third', 'First', 'Second'), dtype=tf.string, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='deck', vocabulary_list=('unknown', 'C', 'G', 'A', 'B', 'D', 'F', 'E'), dtype=tf.string, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='embark_town', vocabulary_list=('Southampton', 'Cherbourg', 'Queenstown', 'unknown'), dtype=tf.string, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='alone', vocabulary_list=('n', 'y'), dtype=tf.string, def

# Creating input function

So why I think we are returing a function in the def (create_input_fn).
when I was creating the model,I got an **error** about a ***callback function***.
So probably we need to pass the function over there to train the model .


---

After printing the values returned by the create_input_fn for training and testing data  I got two different values so maybe it just stores the parameters and the procedure in the stack and executes this when called as a callback function during training using the tf estimator .



In [0]:
def create_input_fn(data_df,label_df,num_epochs=10,shuffle=True,batch_size=32):
  def input_func():
    ds = tf.data.Dataset.from_tensor_slices((dict(data_df),label_df))
    if shuffle:
      ds = ds.shuffle(1000)
    ds = ds.batch(batch_size).repeat(num_epochs)
    return ds
  return input_func

train_input_fn = create_input_fn(dftrain,y_train)
eval_input_fn = create_input_fn(dfeval,y_eval,num_epochs=1,shuffle=False)
print(train_input_fn)
print(eval_input_fn)

<function create_input_fn.<locals>.input_func at 0x7f7fd90b9ae8>
<function create_input_fn.<locals>.input_func at 0x7f7fd90b9b70>


# Creating the model

We are going to use a linear estimator to utilize the linear regression algorithm.
Probably as we are classifying whether the passenger survived or not we use Linear Classifier.

In [0]:
linear_est  = tf.estimator.LinearClassifier(feature_columns=feature_columns)
# We create a linear estimtor by passing the feature columns we created earlie

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpzhspe2jy', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


# Training the model

In [0]:
linear_est.train(train_input_fn)
result = linear_est.evaluate(eval_input_fn)

clear_output()
print(result['accuracy'])

0.7537879


In [0]:
print(result)

{'accuracy': 0.7537879, 'accuracy_baseline': 0.625, 'auc': 0.834527, 'auc_precision_recall': 0.79100823, 'average_loss': 0.47452417, 'label/mean': 0.375, 'loss': 0.46660075, 'precision': 0.6770833, 'prediction/mean': 0.3728531, 'recall': 0.65656567, 'global_step': 200}


Trying to predict stuff

In [0]:
prediction = list(linear_est.predict(eval_input_fn))
clear_output()
print('information of passenger : \n',dfeval.loc[0])
print('Survived : ',y_train[0])
print('Prediction : ')
print(prediction[0]['probabilities'])

information of passenger : 
 sex                          male
age                            35
n_siblings_spouses              0
parch                           0
fare                         8.05
class                       Third
deck                      unknown
embark_town           Southampton
alone                           y
Name: 0, dtype: object
Survived :  0
Prediction : 
[0.9314717  0.06852826]


for survived index probabilty to 1 and for not survived index probabilty to 0,
thats probably because in the dataframe 0 means not survived and 1 means survived.

In the above result the person could not survive and the model predicts that the person did not survive with a probability of 93% and survived with a probability of 6%. 

In [0]:
print(prediction[1])

{'logits': array([-0.77907664], dtype=float32), 'logistic': array([0.31451893], dtype=float32), 'probabilities': array([0.6854811 , 0.31451893], dtype=float32), 'class_ids': array([0]), 'classes': array([b'0'], dtype=object), 'all_class_ids': array([0, 1], dtype=int32), 'all_classes': array([b'0', b'1'], dtype=object)}
